In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import get_custom_objects  # If you need custom objects

# Load the transaction dataset from JSON
data = pd.read_json('processed_data.json', orient='records')  # Adjust filename as needed

# Ensure the JSON has the same columns as the CSV did:
# For example, 'Amount', 'Latitude', 'Longitude', 'Time', 'Is Fraud?' etc.
# If 'Time' is still a string that can be parsed as a datetime, we can convert it:
data['Time'] = pd.to_datetime(data['Time'])

# Preprocessing the dataset
# Selecting relevant columns for RNN training
features = ['Amount', 'Latitude', 'Longitude', 'Time']
target = 'Is Fraud?'

# Extract time-based features (hour, day of week, etc.)
data['Hour'] = data['Time'].dt.hour
data['DayOfWeek'] = data['Time'].dt.dayofweek
data['Day'] = data['Time'].dt.day

# Update features list to include new time-based features
features = ['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']

# Scaling numerical features
scaler = MinMaxScaler()
data[['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']] = scaler.fit_transform(
    data[['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']]
)

# Create feature and target arrays
X = data[features].values
y = data[target].values

# Reshape target for compatibility
y = np.reshape(y, (-1, 1))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=78)

# Reshape the input data for LSTM (samples, timesteps, features)
# Assuming each transaction is independent (timesteps=1)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Building the RNN model
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.9))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.9))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Add EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5)

# Print classification report and AUC
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, y_pred)}")

# Save the model as a .keras file
model.save('fraud_detection_model.keras', save_format='keras')


C:\Users\Thoma\AppData\Local\Temp\ipykernel_25344\624333644.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time'] = pd.to_datetime(data['Time'])




Epoch 1/50


3833/3833 [==============================] - 19s 4ms/step - loss: 0.4969 - accuracy: 0.8045 - val_loss: 0.4652 - val_accuracy: 0.8205
Epoch 2/50
3833/3833 [==============================] - 16s 4ms/step - loss: 0.4814 - accuracy: 0.8122 - val_loss: 0.4659 - val_accuracy: 0.8163
Epoch 3/50
3833/3833 [==============================] - 15s 4ms/step - loss: 0.4779 - accuracy: 0.8134 - val_loss: 0.4655 - val_accuracy: 0.8186
Epoch 4/50
3833/3833 [==============================] - 17s 4ms/step - loss: 0.4770 - accuracy: 0.8137 - val_loss: 0.4640 - val_accuracy: 0.8193
Epoch 5/50
3833/3833 [==============================] - 17s 4ms/step - loss: 0.4750 - accuracy: 0.8145 - val_loss: 0.4635 - val_accuracy: 0.8196
Epoch 6/50
3833/3833 [==============================] - 16s 4ms/step - loss: 0.4738 - accuracy: 0.8152 - val_loss: 0.4634 - val_accuracy: 0.8192
Epoch 7/50
3833/3833 [==============================] - 17s 4ms/step - loss: 0.4740 - accuracy: 0.8149 - val_loss: 0.4629 - va

In [3]:
model.save('fraud_detection_model.h5')


c:\Users\Thoma\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load the dataset
data = pd.read_csv('mock_transactions.csv')

# Drop sensitive features
data = data.drop(['Card Number', 'CVV', 'Expires'], axis=1)

# Encode categorical features
label_encoders = {}
for col in ['Errors?', 'Has Chip', 'City', 'State']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Extract time-based features from 'Time'
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S', errors='coerce')
data['Hour'] = data['Time'].dt.hour

# Handle mixed date formats for 'Date'
data['Date'] = pd.to_datetime(data['Date'], format='mixed', errors='coerce')
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Day'] = data['Date'].dt.day

# Update feature list
# Update feature list to exclude certain columns
features = ['Amount', 'Latitude', 'Longitude', 'Hour', 'DayOfWeek', 'Day']  # Removed specified columns

# Separate features and target
X = data[features]
y = data[target]

# Normalize numerical features
scaler = StandardScaler()
X.loc[:, ['Amount', 'Latitude', 'Longitude']] = scaler.fit_transform(X[['Amount', 'Latitude', 'Longitude']])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=17)

# Define hyperparameters grid
param_grid = {
    'n_estimators': [500, 100],  # Fewer options
    'max_depth': [50, 60],       # Limit depth options
    'min_samples_split': [5],    # Use a single value
    'min_samples_leaf': [2],     # Use a single value
    'max_features': ['sqrt']     # Use a single option
}

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=17)

# Apply GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-8, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Use best model
best_rf_model = grid_search.best_estimator_

# Train the best model
best_rf_model.fit(X_train, y_train)

# Make predictions
y_pred = best_rf_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, y_pred)}")


model.save('fraud_detection_RF.h5')